# Part III
Using the previous two tutorials, please answer the following using an encorder-decoder approach and an LSTM compared approach.

Please create a transformer-based classifier for English name classification into male or female.

There are several datasets for name for male or female classification. In subseuqent iterations, this could be expanded to included more classifications.

Below is the source from NLTK, which only has male and female available but could be used for the purposes of this assignment.

```
names = nltk.corpus.names
names.fileids()
['female.txt', 'male.txt']
male_names = names.words('male.txt')
female_names = names.words('female.txt')
[w for w in male_names if w in female_names]
['Abbey', 'Abbie', 'Abby', 'Addie', 'Adrian', 'Adrien', 'Ajay', 'Alex', 'Alexis',
'Alfie', 'Ali', 'Alix', 'Allie', 'Allyn', 'Andie', 'Andrea', 'Andy', 'Angel',
'Angie', 'Ariel', 'Ashley', 'Aubrey', 'Augustine', 'Austin', 'Averil', ...]
```

In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.3 MB/s eta 0:00:00


In [3]:
import numpy as np
import nltk
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


In [4]:
nltk.download('names')

[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Unzipping corpora/names.zip.


True

In [5]:

names = nltk.corpus.names

names.fileids()

male_names = names.words('male.txt')
female_names = names.words('female.txt')
k=[w for w in male_names if w in female_names]
print(k)



['Abbey', 'Abbie', 'Abby', 'Addie', 'Adrian', 'Adrien', 'Ajay', 'Alex', 'Alexis', 'Alfie', 'Ali', 'Alix', 'Allie', 'Allyn', 'Andie', 'Andrea', 'Andy', 'Angel', 'Angie', 'Ariel', 'Ashley', 'Aubrey', 'Augustine', 'Austin', 'Averil', 'Barrie', 'Barry', 'Beau', 'Bennie', 'Benny', 'Bernie', 'Bert', 'Bertie', 'Bill', 'Billie', 'Billy', 'Blair', 'Blake', 'Bo', 'Bobbie', 'Bobby', 'Brandy', 'Brett', 'Britt', 'Brook', 'Brooke', 'Brooks', 'Bryn', 'Cal', 'Cam', 'Cammy', 'Carey', 'Carlie', 'Carlin', 'Carmine', 'Carroll', 'Cary', 'Caryl', 'Casey', 'Cass', 'Cat', 'Cecil', 'Chad', 'Chris', 'Chrissy', 'Christian', 'Christie', 'Christy', 'Clair', 'Claire', 'Clare', 'Claude', 'Clem', 'Clemmie', 'Cody', 'Connie', 'Constantine', 'Corey', 'Corrie', 'Cory', 'Courtney', 'Cris', 'Daffy', 'Dale', 'Dallas', 'Dana', 'Dani', 'Daniel', 'Dannie', 'Danny', 'Darby', 'Darcy', 'Darryl', 'Daryl', 'Deane', 'Del', 'Dell', 'Demetris', 'Dennie', 'Denny', 'Devin', 'Devon', 'Dion', 'Dionis', 'Dominique', 'Donnie', 'Donny', 'Do

In [6]:

all_names_combined = male_names + female_names
labels = [0] * len(male_names) + [1] * len(female_names)

train_names, test_names, train_labels, test_labels = train_test_split(all_names_combined, labels, test_size=0.2, random_state=42)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_sequences = tokenizer(train_names, truncation=True, padding=True, return_tensors='tf')
test_sequences = tokenizer(test_names, truncation=True, padding=True, return_tensors='tf')


train_labels = np.array(train_labels)
test_labels = np.array(test_labels)


model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.get_vocab()), output_dim=64, input_length=train_sequences.input_ids.shape[1]))
model.add(LSTM(64))
model.add(Dense(1, activation='sigmoid'))


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


model.fit(train_sequences.input_ids, train_labels, epochs=10, batch_size=32, validation_data=(test_sequences.input_ids, test_labels))


loss, accuracy = model.evaluate(test_sequences.input_ids, test_labels)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Epoch 1/10
199/199 [==============================] - 17s 35ms/step - loss: 0.5678 - accuracy: 0.6988 - val_loss: 0.4913 - val_accuracy: 0.7527
Epoch 2/10
199/199 [==============================] - 2s 9ms/step - loss: 0.3435 - accuracy: 0.8571 - val_loss: 0.5089 - val_accuracy: 0.7520
Epoch 3/10
199/199 [==============================] - 2s 8ms/step - loss: 0.2169 - accuracy: 0.9083 - val_loss: 0.6080 - val_accuracy: 0.7048
Epoch 4/10
199/199 [==============================] - 1s 7ms/step - loss: 0.1728 - accuracy: 0.9193 - val_loss: 0.6519 - val_accuracy: 0.7407
Epoch 5/10
199/199 [==============================] - 2s 8ms/step - loss: 0.1488 - accuracy: 0.9256 - val_loss: 0.6819 - val_accuracy: 0.7413
Epoch 6/10
199/199 [==============================] - 1s 7ms/step - loss: 0.1329 - accuracy: 0.9304 - val_loss: 0.8437 - val_accuracy: 0.7413
Epoch 7/10
199/199 [==============================] - 1s 7ms/step - loss: 0.1214 - accuracy: 0.9327 - val_loss: 0.8779 - val_accuracy: 0.7495
Epoc

In [7]:
train_input_ids = train_sequences['input_ids'].numpy()
train_attention_mask = train_sequences['attention_mask'].numpy()
train_token_type_ids = train_sequences['token_type_ids'].numpy()

test_input_ids = test_sequences['input_ids'].numpy()
test_attention_mask = test_sequences['attention_mask'].numpy()
test_token_type_ids = test_sequences['token_type_ids'].numpy()

train_labels = np.array(train_labels)
test_labels = np.array(test_labels)
input_ids = Input(shape=(train_input_ids.shape[1],), dtype=tf.int32)
attention_mask = Input(shape=(train_attention_mask.shape[1],), dtype=tf.int32)
token_type_ids = Input(shape=(train_token_type_ids.shape[1],), dtype=tf.int32)

bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')
outputs = bert_model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

model = Model(inputs=[input_ids, attention_mask, token_type_ids], outputs=outputs.logits)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

model.fit([train_input_ids, train_attention_mask, train_token_type_ids], train_labels, batch_size=32, epochs=10)

loss, accuracy = model.evaluate([test_input_ids, test_attention_mask, test_token_type_ids], test_labels)
print(f"Test Accuracy: {accuracy:.4f}")

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 7)]          0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 7)]          0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 7)]          0           []                               
                                                                                                  
 tf_bert_for_sequence_classific  TFSequenceClassifie  109483778  ['input_1[0][0]',                
 ation (TFBertForSequenceClassi  rOutput(loss=None,               'input_2[0][0]',            